In [3]:
import pysam
import pandas as pd
import numpy as np


In [112]:
# Get promoter positions

tss = pd.read_csv('./data/RefSeqCurated.170308.bed.CollapsedGeneBounds.hg38.TSS500bp.bed', sep='\t', header = None)

idx_s = []
for _,row in tss.iterrows():
    if '_' in row[0]:
        row[0] = row[0].split('_')[0]
    
    tss_pos = 0
    if row[5]=='+':
        tss_pos = row[1]
    elif row[5]=='-':
        tss_pos = row[2]
        
    
    idx_s.append([row[0],row[1],row[2],row[3],tss_pos])

In [109]:
row

0        chrY
1    23219206
2    23219706
3        DAZ2
4           0
5           +
Name: 24460, dtype: object

In [5]:
# Table of all motifs

all_motifs = 'motifs/HOCOMOCOv11_core_HUMAN_mono_jaspar_format.txt'

from Bio import SeqIO

fasta_sequences = SeqIO.parse(open(all_motifs),'fasta')

motif_ids = []
for seqs in fasta_sequences:
    motif_ids.append(seqs.id)

In [84]:
# Get motifs

def get_motifs(file, motif_idxs):
    tbx = pysam.TabixFile(file)
    
    motif_hits = []
    for i in range(len(motif_idxs)):
        motif_row = motif_idxs[i]
        for hits in tbx.fetch(motif_row[0], motif_row[1], motif_row[2]):
            promoter_features = hits.split('\t')
            promoter_features.append(motif_row[3])
            promoter_features.append(tss_pos)
            motif_hits.append(promoter_features)

    return motif_hits
        
    
file_name = './motifs/hg38.archetype_motifs.v1.0.bed.gz'
        
promoter_motifs = get_motifs(file_name, idx_s)

[W::hts_idx_load3] The index file is older than the data file: ./motifs/hg38.archetype_motifs.v1.0.bed.gz.tbi


In [87]:
promoter_motifs[0]

['chr1',
 '11622',
 '11634',
 'PRDM5',
 '7.3532',
 '-',
 'PRDM5_MOUSE.H11MO.0.A',
 '1',
 'DDX11L1']

In [93]:
promoter_motifs = pd.DataFrame(promoter_motifs, columns = ['chr','start','stop','family','log10p','strand','motif','num','promoter_gene'])

In [95]:
promoter_motifs

,chr,start,stop,family,log10p,strand,motif,num,promoter_gene
0,chr1,11622,11634,PRDM5,7.3532,-,PRDM5_MOUSE.H11MO.0.A,1,DDX11L1
1,chr1,11623,11635,NFAC/2,8.7696,-,NFAC1_HUMAN.H11MO.0.B,1,DDX11L1
2,chr1,11624,11634,FOX/4,7.9906,+,FOXO1_HUMAN.H11MO.0.A,2,DDX11L1
3,chr1,11626,11638,E2F/3,3.5370,+,E2F8_E2F_1,2,DDX11L1
4,chr1,11627,11637,FOX/7,7.4554,+,FOXO4_forkhead_3,3,DDX11L1
...,...,...,...,...,...,...,...,...,...
8435071,chrY,23219696,23219713,KLF/SP/2,9.8894,-,WT1_HUMAN.H11MO.0.C,5,DAZ2
8435072,chrY,23219697,23219717,GC-tract,8.5097,-,ZN341_HUMAN.H11MO.0.C,1,DAZ2
8435073,chrY,23219698,23219712,CTCF,6.2602,-,CTCFL_HUMAN.H11MO.0.A,1,DAZ2
8435074,chrY,23219700,23219716,P53-like/1,5.7037,-,P53_MOUSE.H11MO.0.A,1,DAZ2


In [96]:
promoter_motifs.to_csv('promoter_motif_features.gzip', compression='gzip')


In [97]:
pd.read_csv('promoter_motif_features.gzip',compression='gzip')


,Unnamed: 0,chr,start,stop,family,log10p,strand,motif,num,promoter_gene
0,0,chr1,11622,11634,PRDM5,7.3532,-,PRDM5_MOUSE.H11MO.0.A,1,DDX11L1
1,1,chr1,11623,11635,NFAC/2,8.7696,-,NFAC1_HUMAN.H11MO.0.B,1,DDX11L1
2,2,chr1,11624,11634,FOX/4,7.9906,+,FOXO1_HUMAN.H11MO.0.A,2,DDX11L1
3,3,chr1,11626,11638,E2F/3,3.5370,+,E2F8_E2F_1,2,DDX11L1
4,4,chr1,11627,11637,FOX/7,7.4554,+,FOXO4_forkhead_3,3,DDX11L1
...,...,...,...,...,...,...,...,...,...,...
8435071,8435071,chrY,23219696,23219713,KLF/SP/2,9.8894,-,WT1_HUMAN.H11MO.0.C,5,DAZ2
8435072,8435072,chrY,23219697,23219717,GC-tract,8.5097,-,ZN341_HUMAN.H11MO.0.C,1,DAZ2
8435073,8435073,chrY,23219698,23219712,CTCF,6.2602,-,CTCFL_HUMAN.H11MO.0.A,1,DAZ2
8435074,8435074,chrY,23219700,23219716,P53-like/1,5.7037,-,P53_MOUSE.H11MO.0.A,1,DAZ2


In [8]:
family_ids = promoter_motifs['family'].unique()

In [61]:
promoter_motifs['motif'].str

## Should i be working on only the H11M0 family motif and then link it back to all the relevant TFs?

array(['PRDM5', 'NFAC/2', 'FOX/4', 'E2F/3', 'FOX/7', 'ZNF53', 'FOX/5',
       'NFI/1', 'NFI/3', 'GRHL', 'Ebox/CAGCTG', 'Ebox/CACGTG/2', 'SNAI2',
       'Ebox/CACCTG', 'HAND1', 'GFI', 'NR/11', 'INSM1', 'KLF/SP/2',
       'GC-tract', 'TFAP2/1', 'RUNX/1', 'AP1/2', 'ZFX', 'KLF/SP/1',
       'Ebox/CATATG', 'Ebox/CAGATGG', 'ZNF384/2', 'NFAT/2', 'HD/2',
       'SPDEF/1', 'ETS/1', 'NFKB/1', 'ZNF335', 'LEF1', 'TCF/LEF',
       'ZNF708', 'SIX/2', 'MAF', 'NR/12', 'IRF/2', 'NR/3', 'SPI',
       'ZNF354', 'ZNF28', 'FOX/1', 'ETS/2', 'MIES', 'NR/16', 'IRF/1',
       'NR/1', 'NR/13', 'ZNF257', 'ZNF667', 'HD/22', 'NR/17', 'NR/15',
       'Ebox/CACGTG/1', 'NR/19', 'TBX/3', 'SMAD', 'SIX/1', 'BCL6/2',
       'STAT/2', 'RFX/3', 'HSF', 'ZNF134', 'ZNF528', 'P53-like/1',
       'MYB/5', 'HIC/1', 'ZNF549', 'SREBF1', 'ZNF524', 'ZIC', 'ZNF143',
       'SMARCA5', 'NFAT/1', 'SOX/1', 'TFAP2/2', 'RFX/1', 'GLIS', 'E2F/2',
       'GLI', 'NR/18', 'ZNF320', 'ZNF331', 'PLAG1', 'HEN1', 'ZNF547',
       'CTCF', 'EGR', 'PAX

In [67]:
promoter_motif_names = promoter_motifs['motif'].unique()

In [46]:

len(np.intersect1d(motif_ids, motif_names))
# motif_ids

401

In [66]:
for x in promoter_motif_names:
    if 'ALX1' in x:
        print(x)

ALX1_HUMAN.H11MO.0.B


In [11]:
promoter_motifs[promoter_motifs['motif'] == 'ALX1_HUMAN.H11MO.0.B']

,chr,start,stop,family,log10p,strand,motif,n
1337,chr1,68853,68864,HD/8,8.6131,+,ALX1_HUMAN.H11MO.0.B,26
347646,chr1,68051427,68051438,HD/8,8.5453,-,ALX1_HUMAN.H11MO.0.B,14
352962,chr1,74235496,74235507,HD/8,8.2471,-,ALX1_HUMAN.H11MO.0.B,2
467360,chr1,143477097,143477108,HD/8,8.1934,-,ALX1_HUMAN.H11MO.0.B,2
497729,chr1,152811934,152811945,HD/8,7.7117,-,ALX1_HUMAN.H11MO.0.B,2
...,...,...,...,...,...,...,...,...
8257734,chrX,101407996,101408007,HD/8,7.4901,+,ALX1_HUMAN.H11MO.0.B,2
8257848,chrX,101407996,101408007,HD/8,7.4901,+,ALX1_HUMAN.H11MO.0.B,2
8348234,chrX,139965441,139965452,HD/8,7.6482,-,ALX1_HUMAN.H11MO.0.B,2
8421672,chrY,6361254,6361265,HD/8,8.7770,-,ALX1_HUMAN.H11MO.0.B,12


In [12]:
promoter_motifs[promoter_motifs['motif'] == 'ALX4_homeodomain_1']

,chr,start,stop,family,log10p,strand,motif,n
382235,chr1,92792257,92792268,HD/8,7.9743,-,ALX4_homeodomain_1,8
555053,chr1,159289578,159289589,HD/8,7.6279,-,ALX4_homeodomain_1,5
918897,chr2,43227348,43227359,HD/8,7.8862,-,ALX4_homeodomain_1,2
1163995,chr2,149859058,149859069,HD/8,7.3742,+,ALX4_homeodomain_1,1
1209407,chr2,174325871,174325882,HD/8,7.7177,+,ALX4_homeodomain_1,1
1226705,chr2,178777101,178777112,HD/8,7.3580,+,ALX4_homeodomain_1,1
1413464,chr3,9249810,9249821,HD/8,7.4433,+,ALX4_homeodomain_1,1
1435683,chr3,12545307,12545318,HD/8,7.5156,+,ALX4_homeodomain_1,1
2167173,chr4,107893383,107893394,HD/8,8.2708,+,ALX4_homeodomain_1,6
2510411,chr5,119037730,119037741,HD/8,7.3384,-,ALX4_homeodomain_1,1


In [1]:
alxlist = ['ALX1_HUMAN.H11MO.0.B'
'ALX4_homeodomain_1'
'ALX3_MA0634.1'
'ALX3_homeodomain_3']

In [69]:
for x in promoter_motif_names:
    if 'ALX' in x:
        print(x)

ALX1_HUMAN.H11MO.0.B
ALX4_homeodomain_1
ALX3_MA0634.1
ALX3_homeodomain_3


In [14]:
promoter_motifs[promoter_motifs['motif']=='ALX3_homeodomain_3']

,chr,start,stop,family,log10p,strand,motif,n
777160,chr1,244053441,244053452,HD/8,7.5677,+,ALX3_homeodomain_3,8
1043019,chr2,96035428,96035439,HD/8,8.1053,+,ALX3_homeodomain_3,7
1173590,chr2,158101990,158102001,HD/8,8.2239,-,ALX3_homeodomain_3,20
1772643,chr3,139233658,139233669,HD/8,8.3897,-,ALX3_homeodomain_3,11
3073760,chr6,113650209,113650220,HD/8,7.5677,-,ALX3_homeodomain_3,5
3681731,chr8,17895547,17895558,HD/8,8.0773,+,ALX3_homeodomain_3,7
3997796,chr9,5304478,5304489,HD/8,8.2239,-,ALX3_homeodomain_3,22
3998001,chr9,5339644,5339655,HD/8,8.2239,-,ALX3_homeodomain_3,22
5313435,chr12,46826189,46826200,HD/8,8.2239,+,ALX3_homeodomain_3,20


In [71]:
promoter_family_names = promoter_motifs['family'].unique()

In [73]:
for x in promoter_family_names:
    if 'ALX' in x:
        print(x)

In [ ]:
### Grouping TFs by families



In [30]:
motifs_grouping = promoter_motifs.copy()
motifs_grouping = motifs_grouping.drop_duplicates(['motif'])



In [31]:
motifs_grouping['gene'] = motifs_grouping['motif'].str.split('_').str[0]



In [35]:
motifs_grouping = motifs_grouping[['family','gene','motif']].reset_index(drop=True)


In [37]:
motifs_grouping.to_csv('motif_grouping_from_promoters_HOCOMOCOv11_FIMO.csv')


In [41]:
motifs_grouping = pd.read_csv('./motif_grouping_from_promoters_HOCOMOCOv11_FIMO.csv',index_col=0)

In [42]:
motifs_grouping

,family,gene,motif
0,PRDM5,PRDM5,PRDM5_MOUSE.H11MO.0.A
1,NFAC/2,NFAC1,NFAC1_HUMAN.H11MO.0.B
2,FOX/4,FOXO1,FOXO1_HUMAN.H11MO.0.A
3,E2F/3,E2F8,E2F8_E2F_1
4,FOX/7,FOXO4,FOXO4_forkhead_3
...,...,...,...
1622,HD/2,MSX1,MSX1_homeodomain_2
1623,HD/2,EMX1,EMX1_MA0612.1
1624,HD/2,PDX1,PDX1_MA0132.2
1625,FOX/5,FOXJ2,FOXJ2_forkhead_2


In [98]:
promoter_motifs.group_by('promoter_gene')

,chr,start,stop,family,log10p,strand,motif,num,promoter_gene
0,chr1,11622,11634,PRDM5,7.3532,-,PRDM5_MOUSE.H11MO.0.A,1,DDX11L1
1,chr1,11623,11635,NFAC/2,8.7696,-,NFAC1_HUMAN.H11MO.0.B,1,DDX11L1
2,chr1,11624,11634,FOX/4,7.9906,+,FOXO1_HUMAN.H11MO.0.A,2,DDX11L1
3,chr1,11626,11638,E2F/3,3.5370,+,E2F8_E2F_1,2,DDX11L1
4,chr1,11627,11637,FOX/7,7.4554,+,FOXO4_forkhead_3,3,DDX11L1
...,...,...,...,...,...,...,...,...,...
8435071,chrY,23219696,23219713,KLF/SP/2,9.8894,-,WT1_HUMAN.H11MO.0.C,5,DAZ2
8435072,chrY,23219697,23219717,GC-tract,8.5097,-,ZN341_HUMAN.H11MO.0.C,1,DAZ2
8435073,chrY,23219698,23219712,CTCF,6.2602,-,CTCFL_HUMAN.H11MO.0.A,1,DAZ2
8435074,chrY,23219700,23219716,P53-like/1,5.7037,-,P53_MOUSE.H11MO.0.A,1,DAZ2


In [106]:
mini = promoter_motifs[0:10000]

In [105]:
# Create promoter motif dict
promoter_motifs.groupby(['promoter_gene']).apply(lambda grp: grp.groupby('family')['log10p'].apply(list).to_dict()).to_dict()


{'AGRN': {'AHR': ['8.1698'],
  'AP1/2': ['8.0260'],
  'CREB3/XBP1': ['5.7287'],
  'CTCF': ['6.3550',
   '4.9019',
   '7.2983',
   '6.9655',
   '7.2749',
   '8.5797',
   '9.9563',
   '7.9169',
   '5.9258',
   '6.7740',
   '4.9944',
   '11.2267',
   '7.1261',
   '4.8024',
   '5.5796',
   '8.3806',
   '5.1442',
   '9.1647',
   '7.3505',
   '7.4882',
   '9.2641',
   '5.8657',
   '6.6978',
   '7.3799',
   '5.0356',
   '5.8883',
   '7.0403',
   '9.0875',
   '6.0427',
   '7.4433'],
  'E2F/2': ['8.5720',
   '8.6329',
   '11.7459',
   '11.3031',
   '8.1107',
   '14.0060',
   '8.1492',
   '7.9858',
   '9.2427',
   '7.5089',
   '8.1222',
   '8.5904',
   '11.8041',
   '8.4178',
   '13.1070',
   '7.9786',
   '7.4116',
   '7.8020',
   '9.4926',
   '13.7992',
   '11.8353',
   '10.4053',
   '9.5528',
   '9.0277',
   '8.3249',
   '7.3613',
   '8.5646',
   '7.6511',
   '8.2355',
   '7.6070',
   '7.8137',
   '9.6542',
   '9.0919'],
  'E2F/4': ['7.4961'],
  'EGR': ['6.3963',
   '13.3873',
   '8.0460',
   

In [101]:
mini.groupby(['promoter_gene']).apply(lambda grp: grp.groupby('family')['family'].count().to_dict()).to_dict()


{'AGRN': {'AHR': 1,
  'AP1/2': 1,
  'CREB3/XBP1': 1,
  'CTCF': 30,
  'E2F/2': 33,
  'E2F/4': 1,
  'EGR': 15,
  'ETS/1': 4,
  'ETS/2': 33,
  'EWSR1/FLI1': 4,
  'Ebox/CACCTG': 3,
  'Ebox/CACGTG/1': 5,
  'Ebox/CACGTG/2': 13,
  'Ebox/CAGATGG': 5,
  'Ebox/CAGCTG': 8,
  'FOX/7': 1,
  'GATA': 1,
  'GC-tract': 241,
  'GLI': 7,
  'GLIS': 2,
  'GMEB2/2': 2,
  'HD/12': 1,
  'HEN1': 7,
  'HIC/1': 1,
  'HIF': 1,
  'HINFP1/1': 4,
  'HINFP1/3': 1,
  'INSM1': 2,
  'IRF/2': 12,
  'KAISO': 1,
  'KLF/SP/1': 25,
  'KLF/SP/2': 243,
  'LEF1': 2,
  'MBD2': 12,
  'MECP2': 6,
  'MFZ1': 5,
  'MTF1': 1,
  'MYB/5': 1,
  'MZF1': 13,
  'NFI/3': 2,
  'NFKB/1': 1,
  'NR/1': 6,
  'NR/11': 4,
  'NR/12': 5,
  'NR/15': 4,
  'NR/16': 5,
  'NR/17': 2,
  'NR/18': 9,
  'NR/19': 3,
  'NR/3': 34,
  'NRF1': 18,
  'OSR2': 5,
  'P53-like/1': 4,
  'PAX/1': 2,
  'PAX/2': 13,
  'PLAG1': 5,
  'PRDM1': 1,
  'PRDM5': 7,
  'PRDM9': 2,
  'RBPJ': 1,
  'REST/NRSF': 2,
  'RFX/1': 5,
  'RFX/3': 1,
  'RUNX/2': 3,
  'SMAD': 2,
  'SMARCA5': 4,


In [108]:
mini[mini['promoter_gene']=='DDX11L1']

,chr,start,stop,family,log10p,strand,motif,num,promoter_gene
0,chr1,11622,11634,PRDM5,7.3532,-,PRDM5_MOUSE.H11MO.0.A,1,DDX11L1
1,chr1,11623,11635,NFAC/2,8.7696,-,NFAC1_HUMAN.H11MO.0.B,1,DDX11L1
2,chr1,11624,11634,FOX/4,7.9906,+,FOXO1_HUMAN.H11MO.0.A,2,DDX11L1
3,chr1,11626,11638,E2F/3,3.5370,+,E2F8_E2F_1,2,DDX11L1
4,chr1,11627,11637,FOX/7,7.4554,+,FOXO4_forkhead_3,3,DDX11L1
...,...,...,...,...,...,...,...,...,...
142,chr1,12095,12112,KLF/SP/2,7.9738,+,ZN281_HUMAN.H11MO.0.A,5,DDX11L1
143,chr1,12099,12108,SREBF1,7.7431,+,SRBP2_HUMAN.H11MO.0.B,2,DDX11L1
144,chr1,12099,12110,KLF/SP/1,7.5075,-,SP8_C2H2_1,6,DDX11L1
145,chr1,12104,12117,SOX/1,9.8535,+,SOX13_MA1120.1,11,DDX11L1
